# [M_00] PROTOCOL: INFRASTRUCTURE INITIALIZATION (HACKATHON EDITION)

**PROJECT:** OMNI-OPERATOR-V1  
**ORGANIZATION:** [OPERATORS' FORGE](https://takzenai-hub.pl)  
**STATUS:** SYSTEM_INITIALIZATION

This notebook serves as the project's starting point. Its purpose is to prepare the development environment and container infrastructure in the project's root directory (ROOT).

**Prerequisites:**
* Installed `uv` (package manager).
* Installed Docker Desktop.

## 1. Operational path correction
Since we're working inside the `notebooks/` folder, we need to shift the operational context one level up, so that configuration files (`pyproject.toml`, `.env`, `docker-compose.yml`) land in the project's main directory.

In [1]:
import os
import sys

# Navigate to ROOT directory (parent of notebooks folder)
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

print(f"LOG: Current working directory (ROOT): {os.getcwd()}")

# Register src path for future imports
sys.path.append(os.path.join(os.getcwd(), "src"))

LOG: Current working directory (ROOT): c:\Users\takze\OneDrive\Pulpit\project\omni-operator-v1


## 2. Project Structure Initialization (Clean Architecture)
We create the directory skeleton for source code (`src`) and system logic.

In [ ]:
# Folders required by system architecture
src_folders = [
    "src/agents",
    "src/core",
    "src/services",
    "src/api"
]

for folder in src_folders:
    os.makedirs(folder, exist_ok=True)
    # Add __init__.py files for Python modules
    if folder.startswith("src"):
        init_file = os.path.join(folder, "__init__.py")
        if not os.path.exists(init_file):
            with open(init_file, "w") as f: pass

print("LOG: The src/ directory structure has been successfully created.")

## 3. Stack and Dependencies Configuration (`pyproject.toml`)
We define the "Sovereignty Stack" using `uv`. We utilize PydanticAI 1.39.0 and Langfuse v2.

In [ ]:
pyproject_content = """
[project]
name = "OMNI-OPERATOR-V1"
version = "0.1.0"
description = "Autonomous Content Distribution Factory - Gemini Hackathon"
readme = "README.md"
requires-python = ">=3.12"
dependencies = [
    "pydantic-ai==1.39.0",
    "fastapi>=0.128.0",
    "uvicorn>=0.40.0",
    "qdrant-client>=1.16.2",
    "langfuse<3.0.0",
    "moviepy>=2.2.1",
    "python-dotenv>=1.2.1",
    "pydantic-settings>=2.4.0",
    "torch",
    "torchvision",
    "torchaudio",
]

[tool.uv]
managed = true
package = false
"""

with open("pyproject.toml", "w", encoding="utf-8") as f:
    f.write(pyproject_content.strip())

print("LOG: The pyproject.toml file has been generated in ROOT.")

## 4. Docker Infrastructure (Stability Protocol)
We generate the `docker-compose.yml` file containing Qdrant database, Postgres, and Langfuse v2 monitoring.

In [ ]:
docker_compose_v2 = """
version: "3.8"
services:
  langfuse:
    image: langfuse/langfuse:2
    container_name: omni_langfuse
    depends_on:
      db: { condition: service_healthy }
    ports:
      - "3000:3000"
    environment:
      - DATABASE_URL=postgresql://postgres:password@db:5432/langfuse
      - NEXTAUTH_URL=http://localhost:3000
      - NEXTAUTH_SECRET=mysecret
      - SALT=mysalt
    restart: always

  db:
    image: postgres:16-alpine
    container_name: omni_db
    environment:
      - POSTGRES_USER=postgres
      - POSTGRES_PASSWORD=password
      - POSTGRES_DB=langfuse
    ports:
      - "5433:5432" 
    volumes:
      - ./postgres_data:/var/lib/postgresql/data
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U postgres"]
      interval: 3s
      timeout: 3s
      retries: 5

  qdrant:
    image: qdrant/qdrant:latest
    container_name: omni_qdrant
    ports:
      - "6333:6333"
    volumes:
      - ./qdrant_data:/qdrant/storage

"""

with open("docker-compose.yml", "w", encoding="utf-8") as f:
    f.write(docker_compose_v2.strip())

print("LOG: docker-compose.yml generated in ROOT.")

## 5. System Configuration and `uv` Synchronization
We create the `src/core/config.py` file and environment variables template.

In [ ]:
# 1. Implementation of configuration module in src/core/config.py
config_code = """
from pydantic_settings import BaseSettings, SettingsConfigDict
from pydantic import Field

class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_file=".env", extra="ignore")

    gemini_api_key: str = Field(alias="GEMINI_API_KEY")
    langfuse_public_key: str = Field(alias="LANGFUSE_PUBLIC_KEY")
    langfuse_secret_key: str = Field(alias="LANGFUSE_SECRET_KEY")
    langfuse_host: str = Field(default="http://localhost:3000", alias="LANGFUSE_HOST")
    qdrant_url: str = Field(default="http://localhost:6333", alias="QDRANT_URL")

settings = Settings()
"""

with open("src/core/config.py", "w", encoding="utf-8") as f:
    f.write(config_code.strip())

# 2. Creating a readable .env.example file in the ROOT directory
env_example_content = """
GEMINI_API_KEY=your_google_api_key_here
LANGFUSE_PUBLIC_KEY=pk-lf-...
LANGFUSE_SECRET_KEY=sk-lf-...
LANGFUSE_HOST=http://localhost:3000
QDRANT_URL=http://localhost:6333
"""

with open(".env.example", "w", encoding="utf-8") as f:
    f.write(env_example_content.strip())

print("LOG: The config.py and .env.example files have been saved correctly in ROOT.")

## STATUS: SETUP COMPLETED

The system now has complete operational infrastructure.

**Next steps:**
1. Manually create a `.env` file in the root directory (based on `.env.example`).
2. Start containers: `docker compose up -d`.
3. Proceed to the `01_MULTIMODAL_ANALYST.ipynb` notebook.